In [32]:
import random
 
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

import keras.utils
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop


In [33]:
path_to_file = keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

# we increase peformance  by using only lowercase
text = open(path_to_file, 'rb').read().decode(encoding='utf-8').lower()

In [34]:
# we only use this much because we dont have the necessary computational power to train the entire dataset


# sort basically sorts it and set basically just saves it as unique text.
characters = sorted(set(text))

# enumerate iterates over a list and returns the list item with an index for example -> 1, fruit
char_to_index = dict((c,i) for i, c in enumerate(characters))
index_to_char = dict((i,c) for i, c in enumerate(characters)) 

print(char_to_index)
print(index_to_char)


{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'a', 14: 'b', 15: 'c', 16: 'd', 17: 'e', 18: 'f', 19: 'g', 20: 'h', 21: 'i', 22: 'j', 23: 'k', 24: 'l', 25: 'm', 26: 'n', 27: 'o', 28: 'p', 29: 'q', 30: 'r', 31: 's', 32: 't', 33: 'u', 34: 'v', 35: 'w', 36: 'x', 37: 'y', 38: 'z'}


In [36]:
seq_length = 40
step_size = 3

# basically sentences and next_characters are the X & Y
# which means sentences are the Input data and the next_characters are the target data

sentences = []
next_characters = []

for i in range(0, len(text) - seq_length, step_size):
    sentences.append(text[i:i+seq_length])
    next_characters.append(text[i+seq_length])

In [37]:
X = np.zeros((len(sentences), seq_length, len(characters)), dtype=bool)
Y = np.zeros((len(sentences), len(characters)), dtype=bool)


for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        X[i, t, char_to_index[character]] = 1
    Y[i, char_to_index[next_characters[i]]] = 1 

X    

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False,  True, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [38]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

C:\Users\Owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.fit(X, Y, batch_size=256, epochs=6)



Epoch 1/6
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 87s 59ms/step - loss: 1.5935
Epoch 2/6
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 82s 56ms/step - loss: 1.4987
Epoch 3/6
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 82s 57ms/step - loss: 1.4547
Epoch 4/6
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 86s 59ms/step - loss: 1.4264
Epoch 5/6
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 82s 56ms/step - loss: 1.3948
Epoch 6/6
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 78s 54ms/step - loss: 1.3858


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=textgen-s.model.

In [43]:
model.export('textgen-s.model')

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=textgen-s.model.